In [386]:
import sys as sys
sys.path.append('C:/Users/Fernando-Bluesquare/Desktop/Data Science/data_pipelines-master/src')

import re
import audit.dhis as dhis 
import audit.completeness as cplt
import processing.data_process as dp
from processing.data_aggregation_process import *
import pandas as pd
import psycopg2 as pypg


In [3]:
%load_ext autoreload
%autoreload 2

In [354]:
hivdr = dhis.dhis_instance(dbname, user, host, psswrd)

### Build the total number of patients

In [24]:
# We filter to get a list of matching data names and corresponding uids for the two sources, in the form of the json that we're going to use later on

art_pnls=['PNLS-DRUG-ABC + 3TC + EFV','PNLS-DRUG-ABC + 3TC + EFV sex','PNLS-DRUG-ABC + 3TC + LPV/r','PNLS-DRUG-ABC + 3TC + LPV/r sex','PNLS-DRUG-ABC + 3TC + NVP',
              'PNLS-DRUG-ABC + 3TC + NVP sex', 'PNLS-DRUG-AZT + 3TC + LPV/r', 'PNLS-DRUG-AZT + 3TC + LPV/r sex','PNLS-DRUG-AZT+3TC+EFV', 'PNLS-DRUG-AZT+3TC+EFV sex',
              'PNLS-DRUG-AZT+3TC+NVP', 'PNLS-DRUG-AZT+3TC+NVP sex','PNLS-DRUG-TDF + 3TC + LPV/r', 'PNLS-DRUG-TDF + 3TC + LPV/r sex', 'PNLS-DRUG-TDF + FTC + NVP', 
              'PNLS-DRUG-TDF + FTC + NVP sex','PNLS-DRUG-TDF+ FTC + EFV', 'PNLS-DRUG-TDF+ FTC + EFV sex', 'PNLS-DRUG-TDF+3TC+EFV sex', 'PNLS-DRUG-TDF+3TC+NVP sex','PNLS-DRUG-Autres (à préciser)',
              'PNLS-DRUG-Autres (à préciser) sex']

art_cordaid = ['ABC + 3TC + EFV', 'ABC + 3TC + LPV/r', 'ABC + 3TC + NVP', 'Autres (à préciser)', 'AZT + 3TC + LPV/r', 'AZT+3TC+ EFV',
                 'AZT+3TC+NVP', 'TDF + 3TC + LPV/r', 'TDF + FTC + NVP', 'TDF+ FTC + EFV', 'TDF+3TC+EFV', 'TDF+3TC+NVP', 'TDF+FTC+LPV+rt']

art_pnls_withoutpre=[datel[10:] for datel in art_pnls]

datavar_json={}
for datel in art_cordaid:
    
    pnls_el=[str('PNLS-DRUG-')+ str(matchel) for matchel in art_pnls_withoutpre if ((str(datel) in matchel) and ('sex' in matchel)) ]
    
    if len(pnls_el) >0:
        pnls_el=pnls_el[0]
    else:
        pnls_el=None
        
    dict_row= {'sources':{'cordaid':{'elementname':str(datel),'cat_filter':None},'pnls':{'elementname':pnls_el,'cat_filter':None}},'preferred_source':'cordaid','methods':{'last':'quarterly'}}
    datavar_json.update({str(datel):dict_row})
    
    
datavar_json['AZT+3TC+ EFV']['sources']['pnls']['elementname']='PNLS-DRUG-AZT+3TC+EFV sex'

In [25]:
for key in datavar_json.keys():
    for source in datavar_json[str(key)]['sources'].keys():
        if datavar_json[str(key)]['sources'][str(source)]['elementname']:
            datavar_json[str(key)]['sources'][str(source)]['uid']=hivdr.dataelement.query('name=="'+datavar_json[str(key)]['sources'][str(source)]['elementname']+'"')['uid'].iloc[0]

In [26]:
datavar_json['Patients HIV']= {
                                        'sources':{
                                                   'cordaid':{'elementname':'Nombre de patients encore sous TARV dans la structure',
                                                                'uid':'Yj8caUQs178',
                                                              'cat_filter':'Anciens cas'
                                                            },
                                                    'pnls':{'elementname':'PNLS-ARV-Patients encoresous TARV dans la structure',
                                                                'uid':'Dd2G5zI0o0a',
                                                                'cat_filter':'AC'
                                                           }
                                                  },
                                        'preferred_source':'cordaid',
                                        'methods':{
                                                    'last':'quarterly'
                                                    }
                                        }

In [17]:
def json_to_dict_df(datavar_json,reconcile_only=False):
    from objectpath import Tree
    import pandas as pd
    
    jsonTree=Tree(datavar_json)
    renconcile_dict={}
    dict_of_df={}
    for dataelement in jsonTree.execute('$*').keys():
    #Creation of db

        data_united_df=pd.DataFrame()
        for dbsource in jsonTree.execute('$*["'+str(dataelement)+'"].sources').keys():
            uid_element=jsonTree.execute('$*["'+str(dataelement)+'"].sources["'+str(dbsource)+'"].uid')
            if uid_element:
                
                df = hivdr.get_data(jsonTree.execute('$*["'+str(dataelement)+'"].sources["'+str(dbsource)+'"].uid'))
                df['value'] = pd.to_numeric(df['value'],'integer')
                cat_filter= jsonTree.execute('$*["'+str(dataelement)+'"].sources["'+str(dbsource)+'"].cat_filter')
                                         
                if cat_filter:
                    df_filter = df.catcomboname.str.contains(cat_filter)
                    df_agg = (df[df_filter][['value', 'monthly', 'dataelementid', 'uidorgunit', 'enddate']]
                                     .groupby(by=['uidorgunit', 'dataelementid', 'monthly', 'enddate']).sum().reset_index()
                              )
                else:
                    df_agg = (df[['value', 'monthly', 'dataelementid', 'uidorgunit', 'enddate']]
                                     .groupby(by=['uidorgunit', 'dataelementid', 'monthly', 'enddate']).sum().reset_index()
                              )
            
                df_agg['source']=dbsource
        
                data_united_df=data_united_df.append(df_agg)
        
        reconcile_element_dict={}
    
    
        def reconcilegroupby(groupdf):
            subreconcile_dict={}
            for dbsource in jsonTree.execute('$*["'+str(dataelement)+'"].sources').keys():
                orgdict={dbsource:groupdf.query('source=="'+str(dbsource)+'"')}
                subreconcile_dict.update(orgdict)
                
            reconcile_element_dict[str(groupdf.name)]=subreconcile_dict
            
            reconcile_subdf= dp.measured_serie(subreconcile_dict, 'stock', jsonTree.execute('$*["'+str(dataelement)+'"].preferred_source'))
            reconcile_subdf.reconcile_series()
            return reconcile_subdf.preferred_serie
    
    
        reconcile_element_df=data_united_df.groupby(by=['uidorgunit']).apply(reconcilegroupby).reset_index(drop=True)
        
        if not reconcile_only:
            renconcile_dict[str(dataelement)]={'data_element_dict':reconcile_element_dict,'data_element_reconciledf':reconcile_element_df}
        else:
            renconcile_dict[str(dataelement)]=reconcile_element_df
            
    return renconcile_dict


In [18]:
%%time

dict_df_patients=json_to_dict_df(datavar_json,reconcile_only=True)

Wall time: 10min 5s


In [35]:
def process_agg(dict_df_patients,datavar_json,datelment_list=['value']):
    from objectpath import Tree
    jsonTree=Tree(datavar_json)
    
    dict_of_processed_df_patients={}
    for element_key in dict_df_patients.keys():
        methods_dict=jsonTree.execute('$*["'+str(element_key)+'"].methods')
        for method_key in methods_dict.keys():
        
            pro_df=processing(datanames_list=datelment_list,dat_df=dict_df_patients[element_key])
            processed_df=aggregation(datel_list=datelment_list,df_data=pro_df,method=method_key,agg_period_type=methods_dict[method_key])
            dict_of_processed_df_patients[str(element_key)]={str(method_key):processed_df}
        
    return dict_of_processed_df_patients

In [33]:
%%time
dict_of_processed_df_patients=process_agg(dict_df_patients,datavar_json)

C:/Users/Fernando-Bluesquare/Desktop/Data Science/data_pipelines-master/src\processing\data_aggregation_process.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['monthly']=data.enddate.apply(month_assignment)
C:/Users/Fernando-Bluesquare/Desktop/Data Science/data_pipelines-master/src\processing\data_aggregation_process.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['quarterly']=data.enddate.apply(quarter_assignment)


Wall time: 2min 26s


In [67]:
element_creation_df=data_element_creation_csv(dict_of_processed_df_patients)
element_creation_df.to_csv('./data_element_creation.csv',index=False,encoding ='utf-8')

In [85]:
df_csv=output_csv(dict_of_processed_df_patients,hivdr)
df_csv.to_csv('./data_values_df.csv',index=False,encoding ='utf-8')

## Looking at stock outs

In [107]:
r_exclude =['PNLS-DRUG-CTX 480 / 960 mg ces - Bt 500 ces', 'PNLS-DRUG-CTX 480 mg ces - Bt 1000 ces',
             'PNLS-DRUG-Hepatitis, HBsAg, Determine Kit, 100 Tests','PNLS-DRUG-Hepatitis, HCV, Rapid Device, Serum/Plasma/Whole Blood, kit de 40 Tests',
             'PNLS-DRUG-HIV 1/2, Double Check Gold, Kit de 100 test','PNLS-DRUG-HIV 1+2, Determine Complete, Kit de 100 tests',
             'PNLS-DRUG-HIV 1+2, Uni-Gold HIV, Kit de 20 tests', 'PNLS-DRUG-INH 100 mg; 300 mg - Cés', 'PNLS-DRUG-INH 50mg/5 ml - Sol.Orale',
             'PNLS-DRUG-Syphilis RPR Kit, kit de 100 Tests Determine syph','PNLS-DRUG-CTX 96 mg / ml - Inj']

In [108]:
#Selection of Variables

catlab_sources_dict={'SO':'Sortie','RS':'Nbr de jours RS','EN':'Entrée','SI':'Stock Initial'}
datelemlist_sources_dict={}
for key in catlab_sources_dict.keys():
    c_cordaid = hivdr.dataelement[hivdr.dataelement.name.str.contains(str(key)+' -')]
    cco = hivdr.categoryoptioncombo.categoryoptioncomboid[hivdr.categoryoptioncombo.name.str.contains(catlab_sources_dict[key])].iloc[0]
    pnls_cco_uid = hivdr.categoryoptioncombo.uid[hivdr.categoryoptioncombo.name.str.contains(catlab_sources_dict[key])].iloc[0]
    cc = hivdr.categorycombos_optioncombos.categorycomboid[hivdr.categorycombos_optioncombos.categoryoptioncomboid == cco].iloc[0]
    c_pnls = hivdr.dataelement[hivdr.dataelement.categorycomboid == cc]
    datelemlist_sources_dict[str(key)]={'cordaid':c_cordaid,'pnls':c_pnls,'pnls_uid':pnls_cco_uid} 

In [109]:
datelemlist_sources_dict.keys()

dict_keys(['SO', 'RS', 'EN', 'SI'])

In [110]:
def initial_merge(datelemlist_sources_dict):
    srce_dict=datelemlist_sources_dict
    
    for key in srce_dict.keys():

        srce_dict[key]['cordaid']['stand_name']=srce_dict[key]['cordaid'].name.str.replace(str(key)+' -','')
        srce_dict[key]['cordaid']['stand_name']=srce_dict[key]['cordaid'].stand_name.str.replace(' ','').str.lower()
        srce_dict[key]['pnls']['stand_name']=srce_dict[key]['pnls'].name.str.replace(" ",'')
        srce_dict[key]['pnls']['stand_name']=srce_dict[key]['pnls'].stand_name.str.replace("PNLS-DRUG-",'').str.lower()     
        
        srce_dict[key]['merge_dict_list']= srce_dict[key]['pnls'].merge(srce_dict[key]['cordaid'], on='stand_name', suffixes=['_pnls', '_cordaid'])

        srce_dict[key]['pnls'].columns = ['uid_pnls', 'name_pnls', 'dataelementid_pnls' , 'categorycomboid_pnls', 'stand_name']
        srce_dict[key]['cordaid'].columns = ['uid_cordaid', 'name_cordaid', 'dataelementid_cordaid' , 'categorycomboid_cordaid', 'stand_name']
        
    return srce_dict

In [111]:
datelemlist_processed_sources_dict=initial_merge(datelemlist_sources_dict)

C:\Users\Fernando-Bluesquare\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Fernando-Bluesquare\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\Fernando-Bluesquare\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pa

In [112]:
#Cases of missspelling to be included in the merge dataframe

for key in datelemlist_processed_sources_dict.keys():
    merge_list_df=datelemlist_processed_sources_dict[key]['merge_dict_list']
    list_pnls=datelemlist_processed_sources_dict[key]['pnls']
    list_cordaid=datelemlist_processed_sources_dict[key]['cordaid']
    
    for leftout in ['(200/50 mg)','(300/200 mg)','NVP 50 mg']:
        pnls_leftout_values=list_pnls.drop('stand_name', axis=1)[list_pnls.name_pnls.str.contains(leftout)].reset_index(drop=True)
        coraid_leftout_values=list_cordaid[list_cordaid.name_cordaid.str.contains(leftout)].reset_index(drop=True)
        concat_row=pd.concat([pnls_leftout_values,coraid_leftout_values],axis=1)
        merge_list_df = merge_list_df.append(concat_row)

    datelemlist_processed_sources_dict[key]['merge_dict_list'] = merge_list_df[~merge_list_df.name_pnls.isin(r_exclude)]

C:\Users\Fernando-Bluesquare\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  if __name__ == '__main__':
C:\Users\Fernando-Bluesquare\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Fernando-Bluesquare\Anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [115]:
def get_hivdrable(de_id):
    data = hivdr.get_data(de_id)
    data = data[['dataelementid', 'monthly', 'uidorgunit', 'catcomboid', 'value']]
    data.columns = ['dataElement' , 'monthly' , 'orgUnit' , 'categoryOptionCombo', 'value']
    return data

def datadf_into_meds_source_dict(processed_sources_df,pnls_stock_uid):
    meds_dict ={}
    for line in processed_sources_df.name_cordaid:
        print(line)
        id_cordaid = processed_sources_df.uid_cordaid[processed_sources_df.name_cordaid == line].iloc[0]
        id_pnls = processed_sources_df.uid_pnls[processed_sources_df.name_cordaid == line].iloc[0]
        cordaid_data = get_hivdrable(id_cordaid)
        pnls_data = get_hivdrable(id_pnls)
        pnls_data = pnls_data[pnls_data.categoryOptionCombo == pnls_stock_uid]
        meds_dict[str(line)[5:]]={'cordaid':cordaid_data,'pnls':pnls_data}
    return meds_dict

In [119]:
%%time
large_meds_dict={}
for key in datelemlist_processed_sources_dict.keys():
    rs_meds_dict=datadf_into_meds_source_dict(datelemlist_processed_sources_dict[key]['merge_dict_list'],datelemlist_processed_sources_dict[key]['pnls_uid'])
    large_meds_dict[str(key)]=rs_meds_dict

SO - 3TC 10 mg/ml solution orale - Fl. 240 ml
SO - 3TC 150 mg - 60 ces
SO - ABC 300 mg - 60 ces
SO - ABC/3TC ( 300/150 mg) - 60 ces
SO - ABC /3TC 60/30 mg disp. - 60ces
SO - ABC 60 mg disp - 60 ces
SO - AZT 300 mg - 60 ces
SO - AZT/3TC (300/150 mg) - 60 ces
SO - AZT/3TC 60/30 mg ces disp - 60ces
SO - AZT/3TC/NVP (300/150/200 mg) - 60 ces
SO - AZT/3TC/NVP 60/30/50 mg ces disp - 60 ces
SO - EFV 600 mg - 30 ces
SO - LPV/r 100/25 mg ces - 60 ces
SO - LPV/r 80/20 mg/ml solution orale - Fl. 60 ml
SO - LVP/r (133/33 mg) - 120 ces
SO - NVP 200 mg - 60 ces
SO - TDF 300 mg - 30 ces
SO - TDF/3TC (300/300 mg) - 30 CES
SO - TDF/3TC/EFV (300/300/600 mg) - 30 ces
SO - LPV/r (200/50 mg) - 12O ces
SO - TDF/FTC (300/200 mg) - 30 ces
SO - NVP 50 mg ces pour solution orale - 60 Cés
RS - 3TC 10 mg/ml solution orale - Fl. 240 ml
RS - 3TC 150 mg - 60 ces
RS - ABC 300 mg - 60 ces
RS - ABC/3TC ( 300/150 mg) - 60 ces
RS - ABC /3TC 60/30 mg disp. - 60ces
RS - ABC 60 mg disp - 60 ces
RS - AZT 300 mg - 60 ces
RS -

In [136]:

import re

def pills_per_package(line):
    if re.search('[Cc][EeÉé][Ss]$',line):
        line=line[-7:3]
        line=re.sub('[Oo]|" "','', line)
        return float(line)
    else:
        return None
#Finally I didn't use this one part because a talk I had with Antoine 

def measurement_boxes(meds_label,datadf):
    pppck=pills_per_package
    if pppck:
        datadf.value = pd.to_numeric(datadf.value)
        datadf['boxes']=datadf.value/pppck
    return datadf

In [142]:
def reconcilegroupby(groupdf):
    subreconcile_dict={}
    for dbsource in ['cordaid','pnls']:
        orgdict={dbsource:groupdf.query('source=="'+str(dbsource)+'"')}
        subreconcile_dict.update(orgdict)
            
    reconcile_subdf= dp.measured_serie(subreconcile_dict, 'stock', 'cordaid')
    reconcile_subdf.reconcile_series()
    return reconcile_subdf.preferred_serie

In [152]:
def get_preferes_series_from_meds(large_meds_dict):
    renconcile_dict={}
    for typedata in large_meds_dict.keys():
        renconcile_dict[str(typedata)]={}
        for medicine in large_meds_dict[typedata].keys():
            data_united_df=pd.DataFrame()
            for dbsource in ['cordaid','pnls']:
                df_agg=large_meds_dict[typedata][medicine][dbsource]
                df_agg['source']=str(dbsource)
                data_united_df=data_united_df.append(df_agg)
    

            reconcile_element_df=data_united_df.groupby(by=['orgUnit']).apply(reconcilegroupby).reset_index(drop=True)
            renconcile_dict[str(typedata)].update({str(medicine):reconcile_element_df})
            
    return renconcile_dict

In [155]:
%%time
renconcile_dict_meds=get_preferes_series_from_meds(large_meds_dict)

Wall time: 10min 47s


In [165]:
renconcile_dict_transpose={}

for medicine in renconcile_dict_meds['SO'].keys():
    df_list=[]
    for typedata in renconcile_dict_meds.keys():
        df_row=renconcile_dict_meds[typedata][medicine][['monthly','orgUnit','value']]
        df_row=df_row.rename(index=str, columns={'value':str(typedata),'orgUnit':'uidorgunit'})
        df_list.append(df_row)
    
    unified_df=df_list[0]
    for df_typedata in df_list[1:]:
        unified_df=unified_df.merge(df_typedata,how='outer',on=['monthly','uidorgunit'])
        
    renconcile_dict_transpose[str(medicine)]=unified_df

In [302]:
def mean_consumption_stockout(dict_df_med):
    dict_of_processed_df_patients={}
    for medicine in dict_df_med.keys():
        med_df_processed=processing(datanames_list=['SO','RS','EN','SI'],dat_df=dict_df_med[str(medicine)])
        
        meancm_df=mean_consumation_adjusted(med_df_processed[['uidorgunit','enddate','SO','RS']],data_el=medicine)
        combination_df=aggregation(datel_list=['EN','SI'],df_data=med_df_processed,method='combination',agg_period_type='monthly')
    
        stock_df=meancm_df.merge(combination_df,on=['uidorgunit','period'],how='outer')
        stock_df=stock_df.merge(med_df_processed[['uidorgunit','monthly','RS']].rename(index=str, columns={'monthly':'period'}),on=['uidorgunit','period'],how='outer')
    
        stockout_df=stockout(stock_df,data_elem=str(medicine),type_period='monthly')
            
        dict_of_processed_df_patients[str(medicine)]={'AMC':meancm_df,'combination':combination_df,'stockout':stockout_df}
        
    return dict_of_processed_df_patients

In [334]:
%%time
dict_of_processed_df_patients=mean_consumption_stockout(renconcile_dict_transpose)

C:/Users/Fernando-Bluesquare/Desktop/Data Science/data_pipelines-master/src\processing\data_aggregation_process.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['enddate']=data.monthly.apply(enddate_assignment)
C:/Users/Fernando-Bluesquare/Desktop/Data Science/data_pipelines-master/src\processing\data_aggregation_process.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['quarterly']=data.enddate.apply(quarter_assignment)
C:/Users/Fernando-Bluesquare/Desktop/Data Science/data_pipelines-master/src\processing\data_aggregatio

Wall time: 44min 33s


In [ ]:
def mean_consumption_stockout(dict_df_med):
    dict_of_processed_df_patients={}
    for medicine in dict_df_med.keys():
        med_df_processed=processing(datanames_list=['SO','RS','EN','SI'],dat_df=dict_df_med[str(medicine)])
        
        meancm_df=mean_consumation_adjusted(med_df_processed[['uidorgunit','enddate','SO','RS']],data_el=medicine)
        combination_df=aggregation(datel_list=['EN','SI'],df_data=med_df_processed,method='combination',agg_period_type='monthly')
    
        stock_df=meancm_df.merge(combination_df,on=['uidorgunit','period'],how='outer')
        stock_df=stock_df.merge(med_df_processed[['uidorgunit','monthly','RS']].rename(index=str, columns={'monthly':'period'}),on=['uidorgunit','period'],how='outer')
    
        stockout_df=stockout(stock_df,data_elem=str(medicine),type_period='monthly')
            
        dict_of_processed_df_patients[str(medicine)]={'AMC':meancm_df,'combination':combination_df,'stockout':stockout_df}
        
    return dict_of_processed_df_patients

In [379]:
element_creation_df=data_element_creation_csv(dict_of_processed_df_patients)
element_creation_df.to_csv('./data_element_creation_drugs.csv',index=False,encoding ='utf-8')

In [406]:
hivdr = dhis.dhis_instance(dbname, user, host, psswrd)
df_csv=output_csv(dict_of_processed_df_patients,hivdr)

3TC 10 mg/ml solution orale - Fl. 240 mlAMC
3TC 10 mg/ml solution orale - Fl. 240 mlcombination
3TC 10 mg/ml solution orale - Fl. 240 mlstockout
3TC 150 mg - 60 cesAMC
3TC 150 mg - 60 cescombination
3TC 150 mg - 60 cesstockout
ABC 300 mg - 60 cesAMC
ABC 300 mg - 60 cescombination
ABC 300 mg - 60 cesstockout
ABC/3TC ( 300/150 mg) - 60 cesAMC
ABC/3TC ( 300/150 mg) - 60 cescombination
ABC/3TC ( 300/150 mg) - 60 cesstockout
ABC /3TC 60/30 mg disp. - 60cesAMC
ABC /3TC 60/30 mg disp. - 60cescombination
ABC /3TC 60/30 mg disp. - 60cesstockout
ABC 60 mg disp - 60 cesAMC
ABC 60 mg disp - 60 cescombination
ABC 60 mg disp - 60 cesstockout
AZT 300 mg - 60 cesAMC
AZT 300 mg - 60 cescombination
AZT 300 mg - 60 cesstockout
AZT/3TC (300/150 mg) - 60 cesAMC
AZT/3TC (300/150 mg) - 60 cescombination
AZT/3TC (300/150 mg) - 60 cesstockout
AZT/3TC 60/30 mg ces disp - 60cesAMC
AZT/3TC 60/30 mg ces disp - 60cescombination
AZT/3TC 60/30 mg ces disp - 60cesstockout
AZT/3TC/NVP (300/150/200 mg) - 60 cesAMC
AZT/3